In [1]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline as backend_inline

backend_inline.set_matplotlib_formats("svg")


# Pytorch device specific configuration
# Pytorch Gpu Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)


In [18]:
# Generate data
data = (np.random.randn(50000, 2) * 100)
labels = np.sum(data, axis=1)
print(f'Data Shape: {data.shape}, Label Shape: {labels.shape}')

Data Shape: (50000, 2), Label Shape: (50000,)


In [19]:
# Convert data into tensors
dataT = torch.tensor(data).float()
labelsT = torch.tensor(labels).long()

# train test splits
train_data, test_data_temp, train_labels, test_labels_temp = train_test_split(dataT, labelsT, train_size=.8)
test_data, dev_data, test_labels, dev_labels = train_test_split(
    test_data_temp, test_labels_temp, train_size=0.5
)

# Pytorch datasets
train_dataT = TensorDataset(train_data, train_labels)
dev_dataT = TensorDataset(dev_data, dev_labels)
test_dataT = TensorDataset(test_data, test_labels)

# Dataloader objects
batchSize = 32
train_loader = DataLoader(train_dataT, batch_size=batchSize, shuffle=True, drop_last=True, generator=torch.Generator(device))
dev_loader = DataLoader(dev_dataT, batch_size=dev_dataT.tensors[0].shape[0], generator=torch.Generator(device))
test_loader = DataLoader(test_dataT, batch_size=test_dataT.tensors[0].shape[0], generator=torch.Generator(device))
